In [1]:
import sys
import os
import glob
import re 
import pandas as pd
import numpy as np
from lxml import etree
import matplotlib.pyplot as plt
import requests
import time
from collections import Counter


In [2]:
metadata_books_df = pd.read_csv("./../data/metadata_books_with_tokens_counts.tsv", sep="\t", index_col=0)

In [3]:
metadata_books_df

,index,path,language,title,year,number_verses_translation,number_chapters_translation,number_books_translation,source,file_name_translation,...,xlm_roberta_base_count_parts_tokens_median,xlm_roberta_base_count_parts_tokens_std,xlm_roberta_base_count_unks_mean,xlm_roberta_base_count_unks_median,xlm_roberta_base_count_unks_std,bert_multilingual_cased_count_tokens_mean/text_count_tokens_mean,bert_multilingual_cased_count_parts_tokens_mean/text_count_tokens_mean,bert_multilingual_cased_count_unks_mean/text_count_tokens_mean,mt5_base_count_starting_tokens_mean/text_count_tokens_mean,xlm_roberta_base_count_starting_tokens_mean/text_count_tokens_mean
0,232,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Bible Tosk,0,31102,1189,66,zefania,SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN BIBLE...,...,0.0,0.0,0.477143,0.0,0.686471,6.765773,0.640028,0.112765,0.889763,0.889763
1,232,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Bible Tosk,0,31102,1189,66,zefania,SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN BIBLE...,...,0.0,0.0,0.339044,0.0,0.559234,6.861003,0.659416,0.088058,0.902305,0.902305
2,232,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Bible Tosk,0,31102,1189,66,zefania,SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN BIBLE...,...,0.0,0.0,0.369759,0.0,0.525919,6.657666,0.626605,0.048952,0.922347,0.922347
3,232,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Bible Tosk,0,31102,1189,66,zefania,SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN BIBLE...,...,0.0,0.0,0.434412,0.0,0.637323,7.011441,0.678076,0.069024,0.910137,0.910137
4,232,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Bible Tosk,0,31102,1189,66,zefania,SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN BIBLE...,...,0.0,0.0,0.324601,0.0,0.521476,6.626879,0.623845,0.069149,0.899222,0.899222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11443,105,/versions/Traducción-en-lenguaje-actual-TLA-Bi...,SPA,Traducción en lenguaje actual (TLA),2000,26654,1189,66,biblegateway,Traducción en lenguaje actual (TLA),...,0.0,0.0,0.531646,0.0,0.657022,5.680684,0.305417,0.065217,0.863150,0.863150
11444,105,/versions/Traducción-en-lenguaje-actual-TLA-Bi...,SPA,Traducción en lenguaje actual (TLA),2000,26654,1189,66,biblegateway,Traducción en lenguaje actual (TLA),...,0.0,0.0,0.034188,0.0,0.182493,5.773759,0.355804,0.001315,0.857613,0.857613
11445,105,/versions/Traducción-en-lenguaje-actual-TLA-Bi...,SPA,Traducción en lenguaje actual (TLA),2000,26654,1189,66,biblegateway,Traducción en lenguaje actual (TLA),...,0.0,0.0,0.186047,0.0,0.393750,5.854817,0.289688,0.005427,0.902985,0.902985
11446,105,/versions/Traducción-en-lenguaje-actual-TLA-Bi...,SPA,Traducción en lenguaje actual (TLA),2000,26654,1189,66,biblegateway,Traducción en lenguaje actual (TLA),...,0.0,0.0,0.118012,0.0,0.323629,5.668162,0.271194,0.028465,0.874381,0.874381


In [6]:
metadata_books_df[['index',
 'path',
 'language',
 'title',
 'year','file_name_translation',
 'book',
 'book_file_name',]].head(5)

,index,path,language,title,year,file_name_translation,book,book_file_name
0,232,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Bible Tosk,0,SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN BIBLE...,GEN,GEN_SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN B...
1,232,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Bible Tosk,0,SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN BIBLE...,EXO,EXO_SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN B...
2,232,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Bible Tosk,0,SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN BIBLE...,LEV,LEV_SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN B...
3,232,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Bible Tosk,0,SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN BIBLE...,NUM,NUM_SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN B...
4,232,./../data/bibles/transformed_xml_with_year\SF_...,ALB,Albanian Bible Tosk,0,SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN BIBLE...,DEU,DEU_SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN B...


In [4]:
metadata_books_df.columns.tolist()

['index',
 'path',
 'language',
 'title',
 'year',
 'number_verses_translation',
 'number_chapters_translation',
 'number_books_translation',
 'source',
 'file_name_translation',
 'book',
 'book_file_name',
 'text_count_tokens_mean',
 'text_count_tokens_median',
 'text_count_tokens_std',
 'characters_count_mean',
 'characters_count_median',
 'characters_count_std',
 'bert_cased_count_tokens_mean',
 'bert_cased_count_tokens_median',
 'bert_cased_count_tokens_std',
 'bert_cased_count_starting_tokens_mean',
 'bert_cased_count_starting_tokens_median',
 'bert_cased_count_starting_tokens_std',
 'bert_cased_count_parts_tokens_mean',
 'bert_cased_count_parts_tokens_median',
 'bert_cased_count_parts_tokens_std',
 'bert_cased_count_unks_mean',
 'bert_cased_count_unks_median',
 'bert_cased_count_unks_std',
 'bert_multilingual_cased_count_tokens_mean',
 'bert_multilingual_cased_count_tokens_median',
 'bert_multilingual_cased_count_tokens_std',
 'bert_multilingual_cased_count_starting_tokens_mean',

In [7]:
if not os.path.exists("./../data/books_jointed/"):
    os.makedirs("./../data/books_jointed/")

In [20]:
i = 0

for index, row in metadata_books_df.iterrows():

    df = pd.read_csv("./../data/books_tokenized/" + row["book_file_name"] + ".tsv", sep = "\t", index_col = 0)

    df.insert(loc=2, column = 'book_file_name', value = row["book_file_name"])

    if i == 0:
        jointed_df = df.copy()
    else:
        jointed_df = pd.concat([jointed_df, df])
    
    i += 1


In [21]:
jointed_df.shape

(4782601, 27)

In [22]:
jointed_df

,Unnamed: 0.1,id,book_file_name,text,bert_multilingual_cased,bert_cased,mt5_base,xlm_roberta_base,codebook,text_count_tokens,...,bert_multilingual_cased_count_parts_tokens,bert_multilingual_cased_count_unks,mt5_base_count_tokens,mt5_base_count_starting_tokens,mt5_base_count_parts_tokens,mt5_base_count_unks,xlm_roberta_base_count_tokens,xlm_roberta_base_count_starting_tokens,xlm_roberta_base_count_parts_tokens,xlm_roberta_base_count_unks
0,0,b.GEN.001.001,GEN_SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN B...,Në fillim Perëndia krijoi qiejt dhe tokën.,Në|fill|##im|Per|##ën|##dia|kr|##ijo|##i|q|##i...,N|##ë|fill|##im|Per|##ë|##ndi|##a|k|##ri|##jo|...,▁N|ë|▁fill|im|▁Per|ëndi|a|▁krij|o|i|▁qi|ejt|▁d...,▁Në|▁fillim|▁Per|ën|dia|▁kri|jo|i|▁qi|ej|t|▁dh...,GEN,8.0,...,8.0,1.0,60.0,8.0,0.0,1.0,59.0,8.0,0.0,1.0
1,1,b.GEN.001.002,GEN_SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN B...,"Toka ishte pa trajtë, e zbrazët dhe errësira m...","To|##ka|ishte|pa|tra|##jtë|,|e|z|##bra|##zë|##...","To|##ka|is|##ht|##e|p|##a|t|##raj|##t|##ë|,|e|...","▁To|ka|▁|ishte|▁pa|▁tra|jtë|,|▁|e|▁|zbra|zë|t|...","▁To|ka|▁ishte|▁pa|▁tra|jtë|,|▁e|▁z|bra|z|ët|▁d...",GEN,24.0,...,22.0,0.0,191.0,22.0,0.0,0.0,180.0,22.0,0.0,0.0
2,2,b.GEN.001.003,GEN_SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN B...,Pastaj Perëndia tha: “U bëftë drita!”. Dhe dri...,Past|##aj|Per|##ën|##dia|tha|:|[UNK]|U|bë|##ft...,Past|##aj|Per|##ë|##ndi|##a|th|##a|:|“|U|b|##ë...,▁Pasta|j|▁Per|ëndi|a|▁tha|:|▁“|U|▁|bë|ftë|▁dri...,▁Pastaj|▁Per|ën|dia|▁tha|:|▁“|U|▁bë|ftë|▁d|rit...,GEN,15.0,...,8.0,7.0,79.0,11.0,0.0,1.0,76.0,11.0,0.0,1.0
3,3,b.GEN.001.004,GEN_SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN B...,Dhe Perëndia pa që drita ishte e mirë; dhe Per...,D|##he|Per|##ën|##dia|pa|që|dr|##ita|ishte|e|m...,D|##he|Per|##ë|##ndi|##a|p|##a|q|##ë|d|##rita|...,▁Dhe|▁Per|ëndi|a|▁pa|▁|që|▁drit|a|▁|ishte|▁|e|...,▁Dhe|▁Per|ën|dia|▁pa|▁që|▁d|rita|▁ishte|▁e|▁mi...,GEN,17.0,...,11.0,0.0,112.0,16.0,0.0,0.0,108.0,16.0,0.0,0.0
4,4,b.GEN.001.005,GEN_SF_2019-09-02_ALB_ALSSHQ1994EB_(ALBANIAN B...,Dhe Perëndia e quajti dritën “ditë” dhe errësi...,D|##he|Per|##ën|##dia|e|qua|##jti|dr|##it|##ën...,D|##he|Per|##ë|##ndi|##a|e|q|##ua|##j|##ti|d|#...,▁Dhe|▁Per|ëndi|a|▁|e|▁qu|ajti|▁drit|ën|▁“|ditë...,▁Dhe|▁Per|ën|dia|▁e|▁qua|j|ti|▁dritë|n|▁“|di|t...,GEN,24.0,...,21.0,13.0,165.0,20.0,0.0,1.0,158.0,20.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26649,26649,b.ZEP.003.005,ZEP_Traducción en lenguaje actual (TLA),¡Esos malvados no tienen vergüenza! \n»Yo esto...,¡|Es|##os|mal|##vado|##s|no|tienen|ver|##g|##ü...,¡|E|##so|##s|ma|##l|##va|##dos|no|tie|##nen|ve...,▁¡|Es|os|▁mal|vados|▁no|▁tiene|n|▁ver|güen|za|...,▁¡|Es|os|▁mal|vado|s|▁no|▁tienen|▁ver|gü|enza|...,ZEP,32.0,...,11.0,0.0,191.0,25.0,0.0,0.0,186.0,25.0,0.0,0.0
26650,26650,b.ZEP.003.006,ZEP_Traducción en lenguaje actual (TLA),"»Yo he destruido naciones, \ny he derribado su...","»|Yo|he|des|##tru|##ido|naciones|,|y|he|der|##...","»|Yo|he|des|##tr|##uid|##o|na|##cion|##es|,|y|...","▁»|Yo|▁he|▁destru|ido|▁na|ciones|,|▁|y|▁he|▁de...","▁»|Yo|▁he|▁destru|ido|▁na|ciones|,|▁y|▁he|▁der...",ZEP,35.0,...,10.0,0.0,224.0,30.0,0.0,0.0,218.0,30.0,0.0,0.0
26651,26651,b.ZEP.003.007,ZEP_Traducción en lenguaje actual (TLA),"»Todo esto lo hice por ti, Jerusalén. \nPensé ...","»|Todo|esto|lo|hi|##ce|por|ti|,|Jerusalén|.|Pe...","»|To|##do|est|##o|lo|hi|##ce|p|##or|t|##i|,|Je...","▁»|Todo|▁esto|▁lo|▁|hice|▁por|▁ti|,|▁Jerusal|é...","▁»|To|do|▁esto|▁lo|▁hic|e|▁por|▁ti|,|▁Jer|usa|...",ZEP,34.0,...,11.0,0.0,218.0,30.0,0.0,0.0,213.0,30.0,0.0,0.0
26652,26652,b.ZEP.003.008,ZEP_Traducción en lenguaje actual (TLA),"»Y ahora, como han actuado así, \nya se acerca...","»|Y|ahora|,|como|han|act|##uado|así|,|ya|se|ac...","»|Y|ah|##ora|,|com|##o|ha|##n|act|##uad|##o|as...","▁»|Y|▁|ahora|,|▁como|▁han|▁actua|do|▁|así|,|▁y...","▁»|Y|▁ahora|,|▁como|▁han|▁actua|do|▁así|,|▁ya|...",ZEP,66.0,...,16.0,0.0,365.0,56.0,0.0,0.0,353.0,56.0,0.0,0.0


In [25]:
jointed_df[["id", "book_file_name","text"]].to_parquet("./../data/books_jointed/jointed_books.parquet")